In [ ]:
from textblob import TextBlob
import pandas as pd
import sqlalchemy
#import pyodbc

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('inreality').config('spark.jars.packages','com.microsoft.azure:spark-mssql-connector_2.12:1.2.0').getOrCreate()

server_name = "jdbc:sqlserver://inreality.database.windows.net:1433"
database_name = "data"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "dbo.review"
username = ""
password = "" # Please specify password here

jdbcDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .load()
jdbcDF.show()

In [ ]:

from pyspark.sql.types import *
from pyspark.sql.functions import col
short_df = jdbcDF.select('prodcut_name','reviewText','summary')

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag_sents
 #   return list_words
panda_df = short_df.toPandas()
yes = panda_df['reviewText'].tolist()
list = [str(i) for i in yes]
tagged_texts = pos_tag_sents(map(word_tokenize, list))

In [ ]:
panda_df['pos'] = tagged_texts
def finding_adj(text):
    adj_list = [word_pair[0] for word_pair in text if word_pair[1] == 'JJ' and TextBlob(word_pair[0]).sentiment[0] != 0 and TextBlob(word_pair[0]).sentiment[1] != 0]
    return adj_list
panda_df['adj'] = panda_df['pos'].apply(finding_adj)

In [ ]:
import re
adj_df = panda_df[['prodcut_name','adj']]
# We want to the prouct name to be a particular column, so we need to use index=False to avoid turing them into column
adj_groupby_df = adj_df.groupby('prodcut_name',as_index=False).sum()

def word_cleansing(wordlist):
    new_wordlist = [re.sub("[+.-]",'',word.lower()) for word in wordlist]
    return new_wordlist
adj_groupby_df['adj'] = adj_groupby_df['adj'].apply(word_cleansing)
display(adj_groupby_df)

In [ ]:
from collections import OrderedDict
def reduce_map_on_counting_word_appearance(wordlist):
    order_dict = {}
    for word in wordlist:
        if word not in order_dict.keys():
            order_dict[word] = 1 
        else:
            order_dict[word] += 1
    count_desc_dict = sorted(order_dict.items(),key=lambda x:x[1],reverse=True)
    return count_desc_dict[:10]

def getting_word(text):
    return text[0]
def getting_number(text):
    return text[1]

adj_groupby_df['adj_count'] = adj_groupby_df['adj'].apply(reduce_map_on_counting_word_appearance)
final_adj_group_df = adj_groupby_df[['prodcut_name','adj_count']]
final_adj_explode_df = final_adj_group_df.explode('adj_count')
final_adj_explode_df['word'] = final_adj_explode_df['adj_count'].apply(getting_word)
final_adj_explode_df['appearance'] = final_adj_explode_df['adj_count'].apply(getting_number)
final_adj_df = final_adj_explode_df.drop(columns=['adj_count'],axis=1)
final_adj_df.head(10)

In [ ]:
adj_df_spark = spark.createDataFrame(final_adj_df)
adj_df_table_name = 'dbo.game_adj_top_5'
adj_df_spark.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", adj_df_table_name) \
    .option("user", username) \
    .option("password", password) \
    .save()

In [ ]:
def get_ploarity(text):
    blob = TextBlob(text)
    plority = blob.sentiment[0]
    return plority 
spark_df = short_df.toPandas()
spark_df['score'] = spark_df['reviewText'].apply(get_ploarity)
group_df = spark_df.groupby('prodcut_name',as_index=False).mean().sort_values(by='score', ascending=True)

In [ ]:
group_df_spark = spark.createDataFrame(group_df)
group_df_table_name = 'dbo.product_polarity_score'
group_df_spark.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", group_df_table_name) \
    .option("user", username) \
    .option("password", password) \
    .save()